In [2]:
# Next: Final player choice locked in
# player choice v computer choice comparison
# winner declaration

import cv2
from keras.models import load_model
import numpy as np
import random
import time

def detectClick(event, x, y, flags, param):
    global gameIntro
    global countdown
    global choiceFlag
    if event == cv2.EVENT_LBUTTONDOWN and gamePlay == False:
        gameIntro = True
        countdown = currentTime + 5
    if event == cv2.EVENT_RBUTTONDOWN and gamePlay == True:
        gameIntro = True
        choiceFlag = False
        countdown = currentTime + 5
        gameIntroGenerate()

def createPlayerChoice():
    global solidPlayerChoice
    # choose the best probable value from a buffer of 12 previous lines
    maxValue = np.argmax(prediction)
    playerChosen = (playerChoice[maxValue])

    if len(bufferPlayerChoice) < 12:
        bufferPlayerChoice.append(playerChosen)
    else:
        bufferPlayerChoice.pop(0)
        bufferPlayerChoice.append(playerChosen)
    
    # "Rock" choice is a bit fragile, so accept "Rock" as the best option if 2 or more in buffer
    if bufferPlayerChoice.count("Rock") >= 2:
        solidPlayerChoice = "Rock"
    else:
        solidPlayerChoice = max(bufferPlayerChoice, key = bufferPlayerChoice.count)

def winnerEval(comp, player):
    global cWins
    global pWins
    global round
    global updated
    if comp == player:
        winner = "Draw"
    elif comp == "Rock" and player == "Scissors":
        winner = "C"
    elif comp == "Paper" and player == "Rock":
        winner = "C"
    elif comp == "Scissors" and player == "Paper":
        winner = "C"
    elif player == "-Waiting-":
        winner = "Error"
    else:
        winner = "P"
    if updated == False:
        if winner == "C":
            cWins += 1
            round += 1
        elif winner == "P":
            pWins += 1
            round += 1
        updated = True
    print(round)
    return winner

def placeholderGenerate():
    createPlayerChoice()
    # Make the screen pretty with boxes and labels added to the frame
    # White box background
    cv2.rectangle(frame, (10,430), (440, 350), (255,255,255), -1)
    # Player label
    cv2.putText(frame, "Player", (55,380), 1, 2, (0,0,0))
    # Computer label
    cv2.putText(frame, "Computer", (270,380), 1, 2, (0,0,0))
    # vs separator
    cv2.putText(frame, "V", (190,420), 2, 3, (0,0,0))
    # Create the text of the interpretted choice from the buffer
    cv2.putText(frame, solidPlayerChoice, (20,420), 0, 1, (0,0,0))
    # Green Button
    cv2.rectangle(frame, (175,80), (275, 120), (20,170,20), -1)
    # Begin text
    cv2.putText(frame, "Click", (180,110), 3, 1, (0,0,0))

def randCompChoice():
    global compChoice
    global choiceFlag
    compChoice = random.choice(playerChoice[0:3])

def gameIntroGenerate():
    createPlayerChoice()
    global choiceFlag
    global gameIntro
    global gamePlay
    global updated
    global playerChoice
    updated = False
    if choiceFlag == False:
        randCompChoice()
        choiceFlag = True
    if gameIntro:
        cv2.rectangle(frame, (10,430), (440, 350), (0,255,255), -1)
        # Player label
        cv2.putText(frame, "Player", (55,380), 1, 2, (0,0,0))
        # Computer label
        cv2.putText(frame, "Computer", (270,380), 1, 2, (0,0,0))
        # vs separator
        cv2.putText(frame, str(int(countdown-currentTime)), (190,420), 2, 3, (0,0,0))
        # Create the text of the interpretted choice from the buffer
        cv2.putText(frame, solidPlayerChoice, (20,420), 0, 1, (0,0,0))
        # Create the text of the computer's random choice
        cv2.putText(frame, random.choice(playerChoice[0:3]), (280,420), 0, 1, (0,0,0))
        if int(countdown-currentTime) <= 0:
            gameIntro = False
            gamePlay = True
            gamePlayGenerate()   
    

def gamePlayGenerate():
    global first
    global gamePlay
    global gameIntro
    print("PLAYING!!!!")
        # White box background
    cv2.rectangle(frame, (10,430), (440, 350), (200,200,200), -1)
    # Player label
    cv2.putText(frame, "Player", (55,380), 1, 2, (0,0,0))
    # Computer label
    cv2.putText(frame, "Computer", (270,380), 1, 2, (0,0,0))
    # vs separator
    cv2.putText(frame, "V", (190,420), 2, 3, (0,0,0))
    # Create the text of the interpretted choice from the buffer
    cv2.putText(frame, solidPlayerChoice, (20,420), 4, 1, (0,0,0))
    # Create the text of the computer's random choice
    cv2.putText(frame, compChoice, (280,420), 0, 1, (0,0,0))
    # Green Button
    cv2.rectangle(frame, (50,80), (402, 120), (20,170,20), -1)
    
    # Check to see if there's an overall winner
    if cWins == 3:
        gamePlay = False
        gameIntro = True
        gameOver("Computer")
    elif pWins == 3:
        gameIntro = False
        gamePlay = True
        gameOver("Player")
    else:
        # Actually decide on the winner
        if winnerEval(compChoice, solidPlayerChoice) == "C":
            cv2.putText(frame, "CPU Wins!", (140,110), 3, 1, (0,0,0))
        elif winnerEval(compChoice, solidPlayerChoice) == "P":
            cv2.putText(frame, "Player Wins!", (110,110), 3, 1, (0,0,0))
        elif winnerEval(compChoice, solidPlayerChoice) == "Draw":
            cv2.putText(frame, "Draw", (165,110), 3, 1, (0,0,0))
        else:
            cv2.putText(frame, "Too Slow", (150,110), 3, 1, (0,0,0))
    
def gameOver(winner):
    cv2.rectangle(frame, (50,80), (402, 160), (180,170,50), -1)
    cv2.putText(frame, "GAME OVER", (135,110), 3, 1, (0,0,0))
    cv2.putText(frame, winner + " wins!", (110,150), 3, 1, (0,0,0))
    
# Declare variables
model = load_model('keras_model4.h5')
cap = cv2.VideoCapture(0)
data = np.ndarray(shape=(1, 224, 224, 3), dtype=np.float32)
playerChoice = ["Rock", "Paper", "Scissors", "-Waiting-", "Play Game"]

bufferPlayerChoice = []
solidPlayerChoice = ""
gameIntro = False
gamePlay = False
countdown = 3
compChoice = "EMPTY"
choiceFlag = False
first = True
pWins = 0
cWins = 0
round = 1
updated = False
theEnd = False


while True: 
    # I don't know what ret is. It doesn't appear anywhere else
    ret, frame = cap.read()
    # crop the frame to match teachablemachine's cropped area
    if frame is None:
        print("No Image")
        continue # move to the top of the while loop without any further processing of this iteration
    else:
        frame = frame[32:, 188:]

    # resize the frame to 224 x 224
    resized_frame = cv2.resize(frame, (224, 224), interpolation = cv2.INTER_AREA)
    image_np = np.array(resized_frame)
    normalized_image = (image_np.astype(np.float32) / 127.0) - 1 # Normalize the image
    data[0] = normalized_image
    prediction = model.predict(data)

    # Generate persistent items
    cv2.rectangle(frame, (10,10), (442, 50), (255,255,0), -1)
    message = "Round: " + str(round) + "     " + "Score: " + str(pWins) + ":" + str(cWins)
    cv2.putText(frame, message, (20,40), 1, 2, (0,0,0))
    
    # Output the best choice to the shell
    print(solidPlayerChoice)
    print(compChoice)
    print(time.time())
    currentTime = time.time()

    # Generates appropriate Display overlay
    if theEnd == True:
        pass
    elif gameIntro == True:
        gameIntroGenerate()
    elif gamePlay == True and gameIntro == False:
        gamePlayGenerate()
    else:
        placeholderGenerate()

    # Finally, output the frame window, with a nice title
    cv2.namedWindow("Rock Paper Scissors")
    cv2.imshow("Rock Paper Scissors", frame)

    # Click mouse anywhere to begin
    cv2.setMouseCallback("Rock Paper Scissors", detectClick)

    # Press q to close the window
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
    


            
# After the loop release the cap object
cap.release()
# Destroy all the windows
cv2.destroyAllWindows()

2022-02-02 18:14:14.343433: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-02-02 18:14:14.343448: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-02-02 18:14:16.854096: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-02-02 18:14:16.854403: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-02-02 18:14:16.854466: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.s


EMPTY
1643825658.9203765
-Waiting-
EMPTY
1643825661.2832687
-Waiting-
EMPTY
1643825661.3184197
-Waiting-
EMPTY
1643825661.3585982
-Waiting-
EMPTY
1643825661.393298
-Waiting-
EMPTY
1643825661.4219127
-Waiting-
EMPTY
1643825661.455167
-Waiting-
EMPTY
1643825661.4838285
-Waiting-
EMPTY
1643825661.514564
-Waiting-
EMPTY
1643825661.5510783
-Waiting-
EMPTY
1643825661.580425
-Waiting-
EMPTY
1643825661.6167138
-Waiting-
EMPTY
1643825661.6460536
-Waiting-
EMPTY
1643825661.676009
-Waiting-
EMPTY
1643825661.7075276
-Waiting-
EMPTY
1643825661.7417748
-Waiting-
EMPTY
1643825661.7729561
-Waiting-
EMPTY
1643825661.8061965
-Waiting-
EMPTY
1643825661.8348749
-Waiting-
EMPTY
1643825661.8658361
-Waiting-
EMPTY
1643825661.8997142
-Waiting-
EMPTY
1643825661.9294703
-Waiting-
EMPTY
1643825661.9617176
-Waiting-
EMPTY
1643825662.0160708
-Waiting-
EMPTY
1643825662.050957
-Waiting-
EMPTY
1643825662.0903656
-Waiting-
EMPTY
1643825662.1204703
-Waiting-
EMPTY
1643825662.148462
-Waiting-
EMPTY
1643825662.176512
-W

error: OpenCV(4.5.5) /home/conda/feedstock_root/build_artifacts/libopencv_1641992572944/work/modules/highgui/src/window_QT.cpp:2740: error: (-27:Null pointer) file extension not recognized, please choose between JPG, JPEG, BMP or PNG in function 'saveView'
